In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import pickle as pkl

In [ ]:
import sys
sys.path.append("..")
from common.utility import to_categorical, torch_device
from notebook_utils.utility import class_plot
from notebook_utils.generate_gaussian import generate_gaussian
from notebook_utils.eigan import Encoder, Discriminator
from notebook_utils.eigan_training import centralized
from notebook_utils.bertran import bertran as bertran_gan
from notebook_utils.metrics import centralized as metrics

In [ ]:
device='gpu'
device = torch_device(device=device)
device

# Training

In [ ]:
MAX = 1024
history = {}
NUM_TRIALS = 10
NUM_ITERS = 2001
BATCH_SIZE=1024

for trial in range(NUM_TRIALS):
    for VAR1 in range(2, 9, 2):
        for VAR2 in range(2, 9, 2):
            X, y_1, y_2 = generate_gaussian(VAR1/10, VAR2/10, 1000, 1)
            print('='*80)
            print('VAR1: {}, VAR2: {}, TRIAL: {}'.format(VAR1/10, VAR2/10, trial))
            print('='*80)
            print('X, y_1, y2')
            print(X.shape, y_1.shape, y_2.shape)

            X_train, X_valid, y_1_train, y_1_valid, y_2_train, y_2_valid = train_test_split(
            X, y_1, y_2, test_size=0.2, stratify=pd.DataFrame(
                np.concatenate((y_1, y_2), axis=1)
            ))
            print('X_train, X_valid, y_1_train, y_1_valid, y_2_train, y_2_valid')
            print(X_train.shape, X_valid.shape, y_1_train.shape, y_1_valid.shape,
                  y_2_train.shape, y_2_valid.shape)

            w_1 = np.bincount(y_1_train.flatten())
            w_1 = sum(w_1)/w_1
            w_2 = np.bincount(y_2_train.flatten())
            w_2 = sum(w_2)/w_2
            print('class weights => w1, w2', w_1, w_2)

            scaler = MinMaxScaler()
            X_train = scaler.fit_transform(X_train)
            X_valid = scaler.transform(X_valid)

            y_1_train = to_categorical(y_1_train)
            y_2_train = to_categorical(y_2_train)
            y_1_valid = to_categorical(y_1_valid)
            y_2_valid = to_categorical(y_2_valid)

            X_train = torch.Tensor(X_train).to(device)
            y_1_train = torch.Tensor(y_1_train).to(device)
            y_2_train = torch.Tensor(y_2_train).to(device)

            X_valid = torch.Tensor(X_valid).to(device)
            y_1_valid = torch.Tensor(y_1_valid).to(device)
            y_2_valid = torch.Tensor(y_2_valid).to(device)

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
            class_plot(X_train.cpu(), np.argmax(y_1_train.cpu(), axis=1), 
                       np.argmax(y_2_train.cpu(), axis=1), 'normalized train set', ax1)
            class_plot(X_valid.cpu(), np.argmax(y_1_valid.cpu(), axis=1), 
                       np.argmax(y_2_valid.cpu(), axis=1), 'normalized valid set', ax2)

            print('tensor sizes =>', X_train.shape, X_valid.shape, 
                  y_1_train.shape, y_1_valid.shape, y_2_train.shape, y_2_valid.shape)

            train_dataset = TensorDataset(X_train, y_1_train, y_2_train)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

            alpha = 1
            lr_encd = 0.001
            lr_1 = 0.0001
            lr_2 = 0.0001
            n_iter_gan = NUM_ITERS

            input_size = X_train.shape[1]
            hidden_size = input_size*8
            output_size = input_size


            print("-"*80)
            print('EIGAN Training')
            print("-"*80)
            encoder = centralized(
                X_train, X_valid,
                y_1_train, y_1_valid,
                y_2_train, y_2_valid,
                input_size, hidden_size, output_size,
                alpha, lr_encd, lr_1, lr_2, w_1, w_2,
                train_loader, n_iter_gan, device)
            
            encoder.eval()

            alpha = 1
            lr_encd = 0.0001
            lr_1 = 0.00001
            lr_2 = 0.00001
            n_iter_gan = NUM_ITERS
            d_reps = 1

            input_size = X_train.shape[1]
            hidden_size = input_size*8
            output_size = input_size

            print("-"*80)
            print('Bertran Training')
            print("-"*80)
            bertran = bertran_gan(
                X_train, X_valid,
                y_1_train, y_1_valid,
                y_2_train, y_2_valid,
                input_size, hidden_size, output_size,  
                alpha, lr_encd, lr_1, lr_2, 
                train_loader, n_iter_gan, device)
            
            bertran.eval()

            print("-"*80)
            print('ALLY: BASELINE')
            print("-"*80)
            history['{}_{}_{}_{}_{}'.format(VAR1/10, VAR2/10, 'baseline', 'ally', trial)] = metrics(
                None, input_size, hidden_size, output_size,
                X_train, X_valid, y_1_train, y_1_valid, w_1, device)

            print("-"*80)
            print('ALLY: EIGAN')
            print("-"*80)
            history['{}_{}_{}_{}_{}'.format(VAR1/10, VAR2/10, 'eigan', 'ally', trial)] = metrics(
                encoder, input_size, hidden_size, output_size,
                X_train, X_valid, y_1_train, y_1_valid, w_1, device)

            print("-"*80)
            print('ALLY: BERTRAN')
            print("-"*80)
            history['{}_{}_{}_{}_{}'.format(VAR1/10, VAR2/10, 'bertran', 'ally', trial)] = metrics(
                bertran, input_size, hidden_size, output_size,
                X_train, X_valid, y_1_train, y_1_valid, w_1, device)

            print("-"*80)
            print('ADVERSARY:BASELINE')
            print("-"*80)
            history['{}_{}_{}_{}_{}'.format(VAR1/10, VAR2/10, 'baseline', 'advr', trial)] = metrics(
                None, input_size, hidden_size, output_size,
                X_train, X_valid, y_2_train, y_2_valid, w_2, device)

            print("-"*80)
            print('ADVERSARY:EIGAN')
            print("-"*80)
            history['{}_{}_{}_{}_{}'.format(VAR1/10, VAR2/10, 'eigan', 'advr', trial)] = metrics(
                encoder, input_size, hidden_size, output_size,
                X_train, X_valid, y_2_train, y_2_valid, w_2, device)

            print("-"*80)
            print('ADVERSARY:BERTRAN')
            print("-"*80)
            history['{}_{}_{}_{}_{}'.format(VAR1/10, VAR2/10, 'bertran', 'advr', trial)] = metrics(
                bertran, input_size, hidden_size, output_size,
                X_train, X_valid, y_2_train, y_2_valid, w_2, device)
            
    print("Writing trial {} ...".format(trial))
    pkl.dump(history, open('history_centralized_gaussian_5001epocsh_1024batch.pkl', 'wb'))

In [ ]:

pkl.dump(history, open('history_centralized_gaussian_10_runs.pkl', 'wb'))

# Comparisons

In [ ]:
baseline_ally = []
baseline_ally_mean = []
baseline_advr = []
baseline_advr_mean = []
eigan_ally = []
eigan_ally_mean = []
eigan_advr = []
eigan_advr_mean = []
bertran_ally = []
bertran_ally_mean = []
bertran_advr = []
bertran_advr_mean = []
ddof = 1

for v1 in range(2, 9, 2):
    for v2 in range(2, 9, 2):
        baseline_iter_ally = []
        baseline_iter_advr = []
        eigan_iter_ally = []
        eigan_iter_advr = []
        bertran_iter_ally = []
        bertran_iter_advr = []
        for run in range(0, 10):
            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'baseline', 'ally', run
            )][2]
            baseline_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'baseline', 'advr', run
            )][2]
            baseline_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'eigan', 'ally', run
            )][2]
            eigan_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'eigan', 'advr', run
            )][2]
            eigan_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'bertran', 'ally', run
            )][2]
            bertran_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'bertran', 'advr', run
            )][2]
            bertran_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
            
        baseline_ally += baseline_iter_ally
        baseline_advr += baseline_iter_advr
        eigan_ally += eigan_iter_ally
        eigan_advr += eigan_iter_advr
        bertran_ally += bertran_iter_ally
        bertran_advr += bertran_iter_advr
                
        baseline_iter_ally = np.array(baseline_iter_ally)
        baseline_iter_advr = np.array(baseline_iter_advr)
        eigan_iter_ally = np.array(eigan_iter_ally)
        eigan_iter_advr = np.array(eigan_iter_advr)
        bertran_iter_ally = np.array(bertran_iter_ally)
        bertran_iter_advr = np.array(bertran_iter_advr)
        
        baseline_ally_mean.append(baseline_iter_ally.mean())
        baseline_advr_mean.append(baseline_iter_advr.mean())
        eigan_ally_mean.append(eigan_iter_ally.mean())
        eigan_advr_mean.append(eigan_iter_advr.mean())
        bertran_ally_mean.append(bertran_iter_ally.mean())
        bertran_advr_mean.append(bertran_iter_advr.mean())
              
        print('{}_{}'.format(v1/10, v2/10))
        print('ADVR: {:.2f} +- {:.4f} {:.2f} +- {:.4f} {:.2f} +- {:.4f} => {}'.format(
            baseline_iter_advr.mean(), baseline_iter_advr.var(ddof=ddof),  
            eigan_iter_advr.mean(), eigan_iter_advr.var(ddof=ddof), 
            bertran_iter_advr.mean(), bertran_iter_advr.var(ddof=ddof), 
            eigan_iter_advr.mean() < bertran_iter_advr.mean()
        ))
              
        print('ALLY: {:.2f} +- {:.4f} {:.2f} +- {:.4f} {:.2f} +- {:.4f} => {}'.format(
            baseline_iter_ally.mean(), baseline_iter_ally.var(ddof=ddof),  
            eigan_iter_ally.mean(), eigan_iter_ally.var(ddof=ddof), 
            bertran_iter_ally.mean(), bertran_iter_ally.var(ddof=ddof),
            eigan_iter_ally.mean() > bertran_iter_ally.mean()
        ))

In [ ]:
from scipy import stats

print('Ally: ', stats.ttest_rel(bertran_ally, eigan_ally))
print('Ally Mean: ', stats.ttest_rel(bertran_ally_mean, eigan_ally_mean))
print('Advr: ', stats.ttest_rel(bertran_advr, eigan_advr))
print('Advr Mean: ', stats.ttest_rel(bertran_advr_mean, eigan_advr_mean))

In [ ]:
baseline_ally = []
baseline_ally_mean = []
baseline_advr = []
baseline_advr_mean = []
eigan_ally = []
eigan_ally_mean = []
eigan_advr = []
eigan_advr_mean = []
bertran_ally = []
bertran_ally_mean = []
bertran_advr = []
bertran_advr_mean = []
ddof = 1

for v1 in range(2, 9, 2):
    for v2 in range(2, 9, 2):
        baseline_iter_ally = []
        baseline_iter_advr = []
        eigan_iter_ally = []
        eigan_iter_advr = []
        bertran_iter_ally = []
        bertran_iter_advr = []
        for run in range(0, 10):
            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'baseline', 'ally', run
            )][1]
            baseline_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'baseline', 'advr', run
            )][1]
            baseline_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'eigan', 'ally', run
            )][1]
            eigan_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'eigan', 'advr', run
            )][1]
            eigan_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'bertran', 'ally', run
            )][1]
            bertran_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
            tmp = history['{}_{}_{}_{}_{}'.format(
                v1/10, v2/10, 'bertran', 'advr', run
            )][1]
            bertran_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
            
        baseline_ally += baseline_iter_ally
        baseline_advr += baseline_iter_advr
        eigan_ally += eigan_iter_ally
        eigan_advr += eigan_iter_advr
        bertran_ally += bertran_iter_ally
        bertran_advr += bertran_iter_advr
                
        baseline_iter_ally = np.array(baseline_iter_ally)
        baseline_iter_advr = np.array(baseline_iter_advr)
        eigan_iter_ally = np.array(eigan_iter_ally)
        eigan_iter_advr = np.array(eigan_iter_advr)
        bertran_iter_ally = np.array(bertran_iter_ally)
        bertran_iter_advr = np.array(bertran_iter_advr)
        
        baseline_ally_mean.append(baseline_iter_ally.mean())
        baseline_advr_mean.append(baseline_iter_advr.mean())
        eigan_ally_mean.append(eigan_iter_ally.mean())
        eigan_advr_mean.append(eigan_iter_advr.mean())
        bertran_ally_mean.append(bertran_iter_ally.mean())
        bertran_advr_mean.append(bertran_iter_advr.mean())
              
        print('{}_{}'.format(v1/10, v2/10))
        print('ADVR: {:.2f} +- {:.4f} {:.2f} +- {:.4f} {:.2f} +- {:.4f} => {}'.format(
            baseline_iter_advr.mean(), baseline_iter_advr.var(ddof=ddof),  
            eigan_iter_advr.mean(), eigan_iter_advr.var(ddof=ddof), 
            bertran_iter_advr.mean(), bertran_iter_advr.var(ddof=ddof), 
            eigan_iter_advr.mean() < bertran_iter_advr.mean()
        ))
              
        print('ALLY: {:.2f} +- {:.4f} {:.2f} +- {:.4f} {:.2f} +- {:.4f} => {}'.format(
            baseline_iter_ally.mean(), baseline_iter_ally.var(ddof=ddof),  
            eigan_iter_ally.mean(), eigan_iter_ally.var(ddof=ddof), 
            bertran_iter_ally.mean(), bertran_iter_ally.var(ddof=ddof),
            eigan_iter_ally.mean() > bertran_iter_ally.mean()
        ))

# P-value

In [ ]:
from scipy import stats

print('Ally: ', stats.ttest_rel(bertran_ally, eigan_ally))
print('Ally Mean: ', stats.ttest_rel(bertran_ally_mean, eigan_ally_mean))
print('Advr: ', stats.ttest_rel(bertran_advr, eigan_advr))
print('Advr Mean: ', stats.ttest_rel(bertran_advr_mean, eigan_advr_mean))

# Plots

In [ ]:
history = pkl.load(open('history_centralized_gaussian_5001epocsh_1024batch.pkl', 'rb'))

In [ ]:
v1 = 6
baseline_ally = []
baseline_advr = []
eigan_ally = []
eigan_advr = []
bertran_ally = []
bertran_advr = []

for v2 in range(2, 9, 2):
    baseline_iter_ally = []
    baseline_iter_advr = []
    eigan_iter_ally = []
    eigan_iter_advr = []
    bertran_iter_ally = []
    bertran_iter_advr = []
    for run in range(0, 10):
        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'baseline', 'ally', run
        )][2]
        baseline_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'baseline', 'advr', run
        )][2]
        baseline_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'eigan', 'ally', run
        )][2]
        eigan_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'eigan', 'advr', run
        )][2]
        eigan_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'bertran', 'ally', run
        )][2]
        bertran_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'bertran', 'advr', run
        )][2]
        bertran_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
        
    baseline_ally.append(np.array(baseline_iter_ally))
    baseline_advr.append(np.array(baseline_iter_advr))
    eigan_ally.append(np.array(eigan_iter_ally))
    eigan_advr.append(np.array(eigan_iter_advr))
    bertran_ally.append(np.array(bertran_iter_ally))
    bertran_advr.append(np.array(bertran_iter_advr))
    
    
fig, (ax2, ax1) = plt.subplots(1, 2, figsize=(10, 4))

bp1 = ax1.boxplot(bertran_ally, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.2, 0.4, 0.6, 0.8], showfliers=False, patch_artist=True)
plt.setp(bp1['boxes'], color='cyan')
plt.setp(bp1['medians'], color='black')
bp2 = ax1.boxplot(eigan_ally, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.15, 0.35, 0.55, 0.75], showfliers=False, patch_artist=True)
plt.setp(bp2['boxes'], color='red')
plt.setp(bp2['medians'], color='black')
bp3 = ax1.boxplot(baseline_ally, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.25, 0.45, 0.65, 0.85], showfliers=False, patch_artist=True)
plt.setp(bp3['boxes'], color='orange')
plt.setp(bp3['medians'], color='black')
ax1.set_xlim(left=0, right=1)
ax1.set_title('(b)', y=-0.3)
ax1.set_xlabel('variance along ally label')
ax1.set_ylabel('ally accuracy')
ax1.set_xticks([0.2, 0.4, 0.6, 0.8])
ax1.text(0.3,0.95, 'Higher is better', fontsize=14, color='r')
ax1.legend([bp1["boxes"][0], bp2["boxes"][0], bp3["boxes"][0]], 
           ['Bertran', 'EIGAN', 'Unencoded'], loc='lower left', prop={'size':10})
ax1.grid()

bp1 = ax2.boxplot(bertran_advr, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.2, 0.4, 0.6, 0.8], showfliers=False, patch_artist=True)
plt.setp(bp1['boxes'], color='cyan')
plt.setp(bp1['medians'], color='black')
bp2 = ax2.boxplot(eigan_advr, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.15, 0.35, 0.55, 0.75], showfliers=False, patch_artist=True)
plt.setp(bp2['boxes'], color='red')
plt.setp(bp2['medians'], color='black')
bp3 = ax2.boxplot(baseline_advr, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.25, 0.45, 0.65, 0.85], showfliers=False, patch_artist=True)
plt.setp(bp3['boxes'], color='orange')
plt.setp(bp3['medians'], color='black')
ax2.set_xlim(left=0, right=1)
ax2.set_ylim(top=0.9, bottom=0.5)
ax2.set_title('(a)', y=-0.3)
ax2.set_xlabel('variance along ally label')
ax2.set_ylabel('adversary accuracy')
ax2.set_xticks([0.2, 0.4, 0.6, 0.8])
ax2.text(0.3,0.5, 'Lower is better', fontsize=14, color='r')
ax2.grid()

# v2 = 8
# baseline_ally = []
# baseline_advr = []
# eigan_ally = []
# eigan_advr = []
# bertran_ally = []
# bertran_advr = []

# for v1 in range(2, 9, 2):
#     baseline_iter_ally = []
#     baseline_iter_advr = []
#     eigan_iter_ally = []
#     eigan_iter_advr = []
#     bertran_iter_ally = []
#     bertran_iter_advr = []
#     for run in range(0, 10):
#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'baseline', 'ally', run
#         )][2]
#         baseline_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'baseline', 'advr', run
#         )][2]
#         baseline_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'eigan', 'ally', run
#         )][2]
#         eigan_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'eigan', 'advr', run
#         )][2]
#         eigan_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'bertran', 'ally', run
#         )][2]
#         bertran_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'bertran', 'advr', run
#         )][2]
#         bertran_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
        
#     baseline_ally.append(np.array(baseline_iter_ally))
#     baseline_advr.append(np.array(baseline_iter_advr))
#     eigan_ally.append(np.array(eigan_iter_ally))
#     eigan_advr.append(np.array(eigan_iter_advr))
#     bertran_ally.append(np.array(bertran_iter_ally))
#     bertran_advr.append(np.array(bertran_iter_advr))
    
# bp1 = ax3.boxplot(bertran_ally, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.2, 0.4, 0.6, 0.8], showfliers=False, patch_artist=True)
# plt.setp(bp1['boxes'], color='cyan')
# plt.setp(bp1['medians'], color='black')
# bp2 = ax3.boxplot(eigan_ally, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.15, 0.35, 0.55, 0.75], showfliers=False, patch_artist=True)
# plt.setp(bp2['boxes'], color='red')
# plt.setp(bp2['medians'], color='black')
# bp3 = ax3.boxplot(baseline_ally, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.25, 0.45, 0.65, 0.85], showfliers=False, patch_artist=True)
# plt.setp(bp3['boxes'], color='orange')
# plt.setp(bp3['medians'], color='black')
# ax3.legend([bp1["boxes"][0], bp2["boxes"][0], bp3["boxes"][0]], 
#            ['Bertran', 'EIGAN', 'Unencoded'], loc='upper right', prop={'size':10})
# ax3.set_xlim(left=0, right=1)
# ax3.set_ylim(top=0.95, bottom=0.6)
# ax3.set_title('(d)', y=-0.3)
# ax3.set_xlabel('variance along adversary label')
# ax3.set_ylabel('ally accuracy')
# ax3.set_xticks([0.2, 0.4, 0.6, 0.8])
# ax3.text(0.3,0.9, 'Higher is better', fontsize=14, color='r')
# ax3.grid()

# bp1 = ax4.boxplot(bertran_advr, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.2, 0.4, 0.6, 0.8], showfliers=False, patch_artist=True)
# plt.setp(bp1['boxes'], color='cyan')
# plt.setp(bp1['medians'], color='black')
# bp2 = ax4.boxplot(eigan_advr, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.15, 0.35, 0.55, 0.75], showfliers=False, patch_artist=True)
# plt.setp(bp2['boxes'], color='red')
# plt.setp(bp2['medians'], color='black')
# bp3 = ax4.boxplot(baseline_advr, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.25, 0.45, 0.65, 0.85], showfliers=False, patch_artist=True)
# plt.setp(bp3['boxes'], color='orange')
# plt.setp(bp3['medians'], color='black')
# ax4.set_xlim(left=0, right=1)
# ax4.set_ylim(top=1.01, bottom=0.45)
# ax4.set_title('(c)', y=-0.3)
# ax4.set_xlabel('variance along adversary label')
# ax4.set_ylabel('adversary accuracy')
# ax4.set_xticks([0.2, 0.4, 0.6, 0.8])
# ax4.text(0.3,0.5, 'Lower is better', fontsize=14, color='r')
# ax4.grid()

plt.rcParams.update({'font.size': 14})
fig.subplots_adjust(wspace=0.3)
plt.savefig('gaussian_bertran_comparison_ally.png', bbox_inches='tight', dpi=300)

plt.show()

In [ ]:
v1 = 6
baseline_ally = []
baseline_advr = []
eigan_ally = []
eigan_advr = []
bertran_ally = []
bertran_advr = []

# for v2 in range(2, 9, 2):
#     baseline_iter_ally = []
#     baseline_iter_advr = []
#     eigan_iter_ally = []
#     eigan_iter_advr = []
#     bertran_iter_ally = []
#     bertran_iter_advr = []
#     for run in range(0, 10):
#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'baseline', 'ally', run
#         )][2]
#         baseline_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'baseline', 'advr', run
#         )][2]
#         baseline_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'eigan', 'ally', run
#         )][2]
#         eigan_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'eigan', 'advr', run
#         )][2]
#         eigan_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'bertran', 'ally', run
#         )][2]
#         bertran_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
#         tmp = history['{}_{}_{}_{}_{}'.format(
#             v1/10, v2/10, 'bertran', 'advr', run
#         )][2]
#         bertran_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
        
#     baseline_ally.append(np.array(baseline_iter_ally))
#     baseline_advr.append(np.array(baseline_iter_advr))
#     eigan_ally.append(np.array(eigan_iter_ally))
#     eigan_advr.append(np.array(eigan_iter_advr))
#     bertran_ally.append(np.array(bertran_iter_ally))
#     bertran_advr.append(np.array(bertran_iter_advr))
    
    
fig, (ax4, ax3) = plt.subplots(1, 2, figsize=(10, 4))

# bp1 = ax1.boxplot(bertran_ally, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.2, 0.4, 0.6, 0.8], showfliers=False, patch_artist=True)
# plt.setp(bp1['boxes'], color='cyan')
# plt.setp(bp1['medians'], color='black')
# bp2 = ax1.boxplot(eigan_ally, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.15, 0.35, 0.55, 0.75], showfliers=False, patch_artist=True)
# plt.setp(bp2['boxes'], color='red')
# plt.setp(bp2['medians'], color='black')
# bp3 = ax1.boxplot(baseline_ally, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.25, 0.45, 0.65, 0.85], showfliers=False, patch_artist=True)
# plt.setp(bp3['boxes'], color='orange')
# plt.setp(bp3['medians'], color='black')
# ax1.set_xlim(left=0, right=1)
# ax1.set_title('(b)', y=-0.3)
# ax1.set_xlabel('variance along ally label')
# ax1.set_ylabel('ally accuracy')
# ax1.set_xticks([0.2, 0.4, 0.6, 0.8])
# ax1.text(0.3,0.95, 'Higher is better', fontsize=14, color='r')
# ax1.grid()

# bp1 = ax2.boxplot(bertran_advr, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.2, 0.4, 0.6, 0.8], showfliers=False, patch_artist=True)
# plt.setp(bp1['boxes'], color='cyan')
# plt.setp(bp1['medians'], color='black')
# bp2 = ax2.boxplot(eigan_advr, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.15, 0.35, 0.55, 0.75], showfliers=False, patch_artist=True)
# plt.setp(bp2['boxes'], color='red')
# plt.setp(bp2['medians'], color='black')
# bp3 = ax2.boxplot(baseline_advr, vert=1, whis=1.5, widths=0.05, 
#                   positions=[0.25, 0.45, 0.65, 0.85], showfliers=False, patch_artist=True)
# plt.setp(bp3['boxes'], color='orange')
# plt.setp(bp3['medians'], color='black')
# ax2.set_xlim(left=0, right=1)
# ax2.set_ylim(top=1, bottom=0.45)
# ax2.set_title('(a)', y=-0.3)
# ax2.set_xlabel('variance along ally label')
# ax2.set_ylabel('adversary accuracy')
# ax2.set_xticks([0.2, 0.4, 0.6, 0.8])
# ax2.text(0.3,0.5, 'Lower is better', fontsize=14, color='r')
# ax2.grid()

v2 = 8
baseline_ally = []
baseline_advr = []
eigan_ally = []
eigan_advr = []
bertran_ally = []
bertran_advr = []

for v1 in range(2, 9, 2):
    baseline_iter_ally = []
    baseline_iter_advr = []
    eigan_iter_ally = []
    eigan_iter_advr = []
    bertran_iter_ally = []
    bertran_iter_advr = []
    for run in range(0, 10):
        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'baseline', 'ally', run
        )][2]
        baseline_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'baseline', 'advr', run
        )][2]
        baseline_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'eigan', 'ally', run
        )][2]
        eigan_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'eigan', 'advr', run
        )][2]
        eigan_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))

        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'bertran', 'ally', run
        )][2]
        bertran_iter_ally.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
        tmp = history['{}_{}_{}_{}_{}'.format(
            v1/10, v2/10, 'bertran', 'advr', run
        )][2]
        bertran_iter_advr.append(max(tmp['svm'], tmp['logistic'], tmp['mlp']))
        
    baseline_ally.append(np.array(baseline_iter_ally))
    baseline_advr.append(np.array(baseline_iter_advr))
    eigan_ally.append(np.array(eigan_iter_ally))
    eigan_advr.append(np.array(eigan_iter_advr))
    bertran_ally.append(np.array(bertran_iter_ally))
    bertran_advr.append(np.array(bertran_iter_advr))
    
bp1 = ax3.boxplot(bertran_ally, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.2, 0.4, 0.6, 0.8], showfliers=False, patch_artist=True)
plt.setp(bp1['boxes'], color='cyan')
plt.setp(bp1['medians'], color='black')
bp2 = ax3.boxplot(eigan_ally, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.15, 0.35, 0.55, 0.75], showfliers=False, patch_artist=True)
plt.setp(bp2['boxes'], color='red')
plt.setp(bp2['medians'], color='black')
bp3 = ax3.boxplot(baseline_ally, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.25, 0.45, 0.65, 0.85], showfliers=False, patch_artist=True)
plt.setp(bp3['boxes'], color='orange')
plt.setp(bp3['medians'], color='black')
ax3.legend([bp1["boxes"][0], bp2["boxes"][0], bp3["boxes"][0]], 
           ['Bertran', 'EIGAN', 'Unencoded'], loc='lower right', prop={'size':10})
ax3.set_xlim(left=0, right=1)
ax3.set_ylim(top=0.95, bottom=0.6)
ax3.set_title('(b)', y=-0.3)
ax3.set_xlabel('variance along adversary label')
ax3.set_ylabel('ally accuracy')
ax3.set_xticks([0.2, 0.4, 0.6, 0.8])
ax3.text(0.3,0.9, 'Higher is better', fontsize=14, color='r')
ax3.grid()

bp1 = ax4.boxplot(bertran_advr, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.2, 0.4, 0.6, 0.8], showfliers=False, patch_artist=True)
plt.setp(bp1['boxes'], color='cyan')
plt.setp(bp1['medians'], color='black')
bp2 = ax4.boxplot(eigan_advr, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.15, 0.35, 0.55, 0.75], showfliers=False, patch_artist=True)
plt.setp(bp2['boxes'], color='red')
plt.setp(bp2['medians'], color='black')
bp3 = ax4.boxplot(baseline_advr, vert=1, whis=1.5, widths=0.05, 
                  positions=[0.25, 0.45, 0.65, 0.85], showfliers=False, patch_artist=True)
plt.setp(bp3['boxes'], color='orange')
plt.setp(bp3['medians'], color='black')
ax4.set_xlim(left=0, right=1)
ax4.set_ylim(top=1.01, bottom=0.45)
ax4.set_title('(a)', y=-0.3)
ax4.set_xlabel('variance along adversary label')
ax4.set_ylabel('adversary accuracy')
ax4.set_xticks([0.2, 0.4, 0.6, 0.8])
ax4.text(0.3,0.5, 'Lower is better', fontsize=14, color='r')
ax4.grid()

plt.rcParams.update({'font.size': 14})
fig.subplots_adjust(wspace=0.3)
plt.savefig('gaussian_bertran_comparison_advr.png', bbox_inches='tight', dpi=300)

plt.show()